# Preprocess/CleanUp

In [2]:
#imported python packages
import os
import math
import numpy as np 
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
import glob
import functools
import datetime

In [3]:
# import dataset about train dataset and sample_submission
train = pd.read_csv("train.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [4]:
train

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429
...,...,...
4426,873340274,15695097
4427,1297437712,35659379
4428,694853998,31206935
4429,1886987043,9598270


In [5]:
sample_submission

,segment_id,time_to_eruption
0,1000213997,0
1,100023368,0
2,1000488999,0
3,1001028887,0
4,1001857862,0
...,...,...
4515,996704281,0
4516,997630809,0
4517,998072137,0
4518,998136924,0


In [6]:
# created dataframe about the testing volcanos id
test_data = pd.DataFrame([os.path.basename(f)[:-4] for f in glob.glob('test/*')], columns=["segment_id"])
test_data

,segment_id
0,19158067
1,146639315
2,53465103
3,638074752
4,106558646
...,...
725,1265147
726,115078097
727,577108378
728,108980580


In [7]:
# created dataframe about the training volcanos id
train_data = pd.DataFrame([os.path.basename(f)[:-4] for f in glob.glob('train/*')], columns=["segment_id"])
train_data

,segment_id
0,117864846
1,198187683
2,19334151
3,164402661
4,224267098
...,...
597,130096034
598,140348256
599,442836719
600,88594602


In [8]:
# sorted values depending on the time_to_eruption
train = train.sort_values("time_to_eruption",ascending=False)
train

,segment_id,time_to_eruption
1139,1923243961,49046087
3509,1552761888,48814294
3589,356854390,48727004
3739,1131527270,48522449
4132,1162128945,48387290
...,...,...
1724,1626437563,40492
413,442994108,28696
1145,1957235969,26929
2709,1658693785,25730


In [21]:
# imported one of the training volcano dataset
df = pd.read_csv(f'train/{19334151}.csv')

In [22]:
df

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,-110.0,NaN,-41.0,13.0,149.0,392.0,-34.0,NaN,95.0,264.0
1,-52.0,NaN,-59.0,-17.0,79.0,972.0,-51.0,NaN,59.0,280.0
2,25.0,NaN,-89.0,-69.0,-2.0,912.0,-75.0,NaN,14.0,362.0
3,81.0,NaN,-103.0,-86.0,-47.0,328.0,-85.0,NaN,-14.0,428.0
4,113.0,NaN,-106.0,-86.0,-21.0,-91.0,-70.0,NaN,-5.0,386.0
...,...,...,...,...,...,...,...,...,...,...
59996,163.0,NaN,-134.0,185.0,80.0,-75.0,36.0,NaN,574.0,-845.0
59997,102.0,NaN,-98.0,162.0,56.0,-68.0,22.0,NaN,559.0,-1022.0
59998,38.0,NaN,-48.0,164.0,60.0,-62.0,-15.0,NaN,535.0,-1228.0
59999,0.0,NaN,-13.0,148.0,82.0,-56.0,-52.0,NaN,509.0,-1324.0


### There are some NAN values for sensors 2 and 8. This might be true for some other volcano ids too.

In [18]:
# Converted time_to_eruption to comprehensible time
train['h:m:s'] = (train['time_to_eruption']
                  .apply(lambda x:datetime.timedelta(seconds = x/100)))
train

,segment_id,time_to_eruption,h:m:s
1139,1923243961,49046087,5 days 16:14:20.870000
3509,1552761888,48814294,5 days 15:35:42.940000
3589,356854390,48727004,5 days 15:21:10.040000
3739,1131527270,48522449,5 days 14:47:04.490000
4132,1162128945,48387290,5 days 14:24:32.900000
...,...,...,...
1724,1626437563,40492,0 days 00:06:44.920000
413,442994108,28696,0 days 00:04:46.960000
1145,1957235969,26929,0 days 00:04:29.290000
2709,1658693785,25730,0 days 00:04:17.300000


## Filling the NAN values and extracting features from senosor data.

In [28]:
# created a list of numpy statistical functions
stat_feats = [
    np.nanmin,
    np.nanmax,
    np.nanmedian,
    np.nanmean,
    np.nanstd,
]

# created a function to get the statistical features of the sesor data for each volcano
def get_stat_features(seg):
    
    """
        Returns a Series of different features from the csv.
    """
    
    # Format seg and read csv
    seg = str(seg)

    try:
        seg_df = pd.read_csv(f'train/{seg}.csv')
    except FileNotFoundError:
        seg_df = pd.read_csv(f'test/{seg}.csv')

    seg_df = seg_df.fillna(0)
    
    # Apply each func to each column and append in a Series for each func
    feats_series = [seg_df.apply(f).rename(index=lambda n : f"{n}_{f.__name__}")
                                                                for f in stat_feats]
    
    # Return appended Series
    return functools.reduce(pd.Series.append, feats_series)

In [29]:
# created a dataframe with the features of each volcano's sensor from training dataset using get_stat_features()
train_df = train_data['segment_id'].apply(get_stat_features)

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

...

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

In [34]:
train_df

,sensor_1_nanmin,sensor_2_nanmin,sensor_3_nanmin,sensor_4_nanmin,sensor_5_nanmin,sensor_6_nanmin,sensor_7_nanmin,sensor_8_nanmin,sensor_9_nanmin,sensor_10_nanmin,...,sensor_1_nanstd,sensor_2_nanstd,sensor_3_nanstd,sensor_4_nanstd,sensor_5_nanstd,sensor_6_nanstd,sensor_7_nanstd,sensor_8_nanstd,sensor_9_nanstd,sensor_10_nanstd
0,-1407.0,-4210.0,-1326.0,-1764.0,-1278.0,-1744.0,-2044.0,-2243.0,-1521.0,-3494.0,...,348.518092,421.581562,281.096143,276.654906,168.676062,577.670041,399.186499,547.687085,260.975019,647.819707
1,-509.0,-5078.0,-620.0,-1266.0,-2196.0,-4472.0,-2134.0,-1478.0,-1642.0,-2459.0,...,113.153185,463.843666,121.651476,219.073933,122.909989,562.409158,468.721284,282.091892,227.735263,254.012390
2,-2116.0,0.0,-2653.0,-3785.0,-3336.0,-1792.0,-2292.0,0.0,-3104.0,-7547.0,...,267.036185,0.000000,242.151580,315.977425,208.671741,289.669438,229.103116,0.000000,308.005683,604.834931
3,-1640.0,-15007.0,-1269.0,-1404.0,-1719.0,-1744.0,-1284.0,-4100.0,-1945.0,-4931.0,...,338.815262,866.830526,257.070102,281.781300,232.944795,223.144076,249.329097,243.680349,319.650178,837.884604
4,-1208.0,-2890.0,-1587.0,-1822.0,-1388.0,-4271.0,-3200.0,-7797.0,-1846.0,-4946.0,...,252.330814,385.387826,261.523328,276.030715,154.293986,406.825393,457.665840,558.496913,308.165377,509.506833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,-1955.0,-5845.0,-1264.0,-1390.0,-1058.0,-1702.0,-2007.0,-1843.0,-1504.0,-4924.0,...,374.247910,405.805831,270.460214,314.176935,165.886055,453.948581,397.408789,500.125033,272.936739,641.635572
598,-1199.0,-4566.0,-1597.0,-1714.0,-2546.0,-989.0,-1287.0,-857.0,-1867.0,-3928.0,...,281.371021,578.765112,235.452449,267.211372,259.469482,222.069816,262.296476,234.672983,313.172295,733.310470
599,-3391.0,-2852.0,-3538.0,-1119.0,-837.0,-2814.0,-3556.0,-2607.0,-1233.0,-2477.0,...,393.861151,312.601611,742.416663,250.358440,88.345524,618.793236,522.411907,616.504218,223.204476,364.395521
600,-2014.0,-6331.0,0.0,-1732.0,-1626.0,-1736.0,-1600.0,-1389.0,-1210.0,-3329.0,...,289.027837,502.836668,0.000000,227.693996,168.323730,462.260631,292.234292,294.852997,259.037742,534.569513


In [37]:
# joined train_df with the train dataset
train_df = train.join(train_df)

In [38]:
train_df

,segment_id,time_to_eruption,h:m:s,sensor_1_nanmin,sensor_2_nanmin,sensor_3_nanmin,sensor_4_nanmin,sensor_5_nanmin,sensor_6_nanmin,sensor_7_nanmin,...,sensor_1_nanstd,sensor_2_nanstd,sensor_3_nanstd,sensor_4_nanstd,sensor_5_nanstd,sensor_6_nanstd,sensor_7_nanstd,sensor_8_nanstd,sensor_9_nanstd,sensor_10_nanstd
1139,1923243961,49046087,5 days 16:14:20.870000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3509,1552761888,48814294,5 days 15:35:42.940000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3589,356854390,48727004,5 days 15:21:10.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3739,1131527270,48522449,5 days 14:47:04.490000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4132,1162128945,48387290,5 days 14:24:32.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,1626437563,40492,0 days 00:06:44.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,442994108,28696,0 days 00:04:46.960000,-3120.0,-10881.0,-3182.0,-2882.0,-500.0,-8073.0,-5954.0,...,768.764843,1069.847410,727.041662,688.871537,8.594175,1853.461419,1189.510581,1464.082327,505.825412,1191.737480
1145,1957235969,26929,0 days 00:04:29.290000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2709,1658693785,25730,0 days 00:04:17.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# dropped the NAN values because I am not using all of the volcanoes from train dataset due to storage issues
train_df = train_df.dropna()

In [42]:
# dropped the h:m:s column for better model
train_df = train_df.drop(columns=["h:m:s"])

In [43]:
train_df

,segment_id,time_to_eruption,sensor_1_nanmin,sensor_2_nanmin,sensor_3_nanmin,sensor_4_nanmin,sensor_5_nanmin,sensor_6_nanmin,sensor_7_nanmin,sensor_8_nanmin,...,sensor_1_nanstd,sensor_2_nanstd,sensor_3_nanstd,sensor_4_nanstd,sensor_5_nanstd,sensor_6_nanstd,sensor_7_nanstd,sensor_8_nanstd,sensor_9_nanstd,sensor_10_nanstd
408,1323724621,47861879,-2124.0,-2850.0,-2410.0,-991.0,-1078.0,-2197.0,-4024.0,-2444.0,...,304.556418,336.839651,733.297301,232.474625,97.347796,570.091465,373.866779,574.299883,213.095186,401.136290
24,1581535482,47810569,-2061.0,-7280.0,0.0,-2056.0,-1591.0,-7224.0,-3333.0,-3100.0,...,448.877816,503.899889,0.000000,352.482562,150.162076,2002.897183,533.233834,868.914034,252.359595,630.553190
297,594908465,47775708,-1531.0,-13000.0,-1156.0,-1918.0,-1509.0,-809.0,-1064.0,-938.0,...,342.190608,1258.236329,271.922069,344.210093,230.737410,194.070538,234.100675,222.932381,390.477264,821.316795
359,995669347,47715187,-2130.0,-5213.0,0.0,-1488.0,-1779.0,-6427.0,-5382.0,-13998.0,...,464.925386,393.080345,0.000000,308.931083,142.109170,1710.020864,554.735908,931.369282,328.429315,568.895903
135,895093954,47362238,-1009.0,-4986.0,-931.0,-979.0,-1098.0,-2499.0,-2011.0,-702.0,...,185.900141,549.202012,203.889181,211.165167,128.287458,402.175407,170.829436,186.614540,237.884053,338.188283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,546241772,386910,-1953.0,0.0,-2047.0,-1898.0,-2593.0,-2194.0,-2716.0,0.0,...,275.557088,0.000000,240.808846,399.983434,207.210368,264.248359,291.411985,0.000000,311.783935,615.171584
342,2140509127,376369,-1454.0,-9310.0,-1223.0,-1858.0,-460.0,-3476.0,-4734.0,-2856.0,...,381.743545,943.990710,323.552400,375.720275,23.623666,968.689909,639.425184,780.247065,348.688150,706.266658
98,82371649,273119,-2096.0,-7641.0,-2061.0,-2851.0,-1615.0,-2362.0,-2779.0,-5034.0,...,560.734713,1783.008667,516.670771,676.502377,355.364398,641.416562,523.310707,444.811259,629.645603,1112.261832
413,442994108,28696,-3120.0,-10881.0,-3182.0,-2882.0,-500.0,-8073.0,-5954.0,-5639.0,...,768.764843,1069.847410,727.041662,688.871537,8.594175,1853.461419,1189.510581,1464.082327,505.825412,1191.737480


In [35]:
# created a dataframe with the features of each volcano's sensor from test dataset using get_stat_features()
test_df = test_data['segment_id'].apply(get_stat_features)

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

...

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return functools.reduce(pd.Series.append, feats_series)
/tmp/ipykernel_143/1713493182.py:30: FutureWarning: The series.append method is depr

In [36]:
test_df

,sensor_1_nanmin,sensor_2_nanmin,sensor_3_nanmin,sensor_4_nanmin,sensor_5_nanmin,sensor_6_nanmin,sensor_7_nanmin,sensor_8_nanmin,sensor_9_nanmin,sensor_10_nanmin,...,sensor_1_nanstd,sensor_2_nanstd,sensor_3_nanstd,sensor_4_nanstd,sensor_5_nanstd,sensor_6_nanstd,sensor_7_nanstd,sensor_8_nanstd,sensor_9_nanstd,sensor_10_nanstd
0,-2399.0,-8479.0,-2144.0,-2389.0,0.0,-1907.0,-2725.0,-2276.0,-2724.0,-4591.0,...,537.587907,1699.642640,488.428506,628.862467,0.000000,626.450176,423.136659,366.716709,603.546006,1030.271718
1,-1723.0,0.0,-724.0,-1160.0,0.0,-1652.0,-1448.0,-1485.0,-1021.0,0.0,...,240.370824,0.000000,193.623152,240.484128,0.000000,458.342099,333.692933,383.048540,227.338089,0.000000
2,-1954.0,-12546.0,-1598.0,-2226.0,-2141.0,-1945.0,-2745.0,-4303.0,-2517.0,-3846.0,...,342.889228,1229.756449,317.728830,399.617610,276.604136,519.396151,373.585216,563.451633,428.984636,720.398171
3,0.0,-4235.0,-20655.0,-6658.0,0.0,-9065.0,-4387.0,-5068.0,0.0,0.0,...,0.000000,614.304876,3660.014773,962.320752,0.000000,2342.946778,1206.039887,1572.148256,0.000000,0.000000
4,-25412.0,-32767.0,-32217.0,-27204.0,-10254.0,-10223.0,-15007.0,-14700.0,-30379.0,-32767.0,...,6626.926248,9987.517121,6586.279472,6676.764197,2435.131706,2966.428407,3244.105584,2942.466744,8078.247900,8753.817270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,0.0,-2902.0,-8789.0,-2326.0,0.0,-1434.0,-2194.0,-1860.0,0.0,0.0,...,0.000000,486.778062,1714.410765,364.205085,0.000000,468.322480,323.412116,345.154588,0.000000,0.000000
726,-3924.0,-9948.0,-3716.0,-4771.0,-1192.0,-6078.0,-249.0,-358.0,-3389.0,-4843.0,...,611.803833,825.136816,509.310464,557.989619,184.299317,1662.581413,53.218984,66.156907,540.139436,849.049759
727,0.0,-2052.0,-4692.0,-2369.0,0.0,-1905.0,-1070.0,-1422.0,0.0,0.0,...,0.000000,450.901986,1424.694280,421.877082,0.000000,537.036483,299.404112,353.516468,0.000000,0.000000
728,-2973.0,0.0,-1816.0,-4907.0,-4735.0,-1170.0,-1643.0,0.0,-3476.0,-6436.0,...,280.260756,0.000000,242.735532,341.654619,239.321704,259.951368,233.648477,0.000000,347.004512,634.634993


In [44]:
# joined test_df with the test_data dataset
test_df = test_data.join(test_df)

In [45]:
test_df

,segment_id,sensor_1_nanmin,sensor_2_nanmin,sensor_3_nanmin,sensor_4_nanmin,sensor_5_nanmin,sensor_6_nanmin,sensor_7_nanmin,sensor_8_nanmin,sensor_9_nanmin,...,sensor_1_nanstd,sensor_2_nanstd,sensor_3_nanstd,sensor_4_nanstd,sensor_5_nanstd,sensor_6_nanstd,sensor_7_nanstd,sensor_8_nanstd,sensor_9_nanstd,sensor_10_nanstd
0,19158067,-2399.0,-8479.0,-2144.0,-2389.0,0.0,-1907.0,-2725.0,-2276.0,-2724.0,...,537.587907,1699.642640,488.428506,628.862467,0.000000,626.450176,423.136659,366.716709,603.546006,1030.271718
1,146639315,-1723.0,0.0,-724.0,-1160.0,0.0,-1652.0,-1448.0,-1485.0,-1021.0,...,240.370824,0.000000,193.623152,240.484128,0.000000,458.342099,333.692933,383.048540,227.338089,0.000000
2,53465103,-1954.0,-12546.0,-1598.0,-2226.0,-2141.0,-1945.0,-2745.0,-4303.0,-2517.0,...,342.889228,1229.756449,317.728830,399.617610,276.604136,519.396151,373.585216,563.451633,428.984636,720.398171
3,638074752,0.0,-4235.0,-20655.0,-6658.0,0.0,-9065.0,-4387.0,-5068.0,0.0,...,0.000000,614.304876,3660.014773,962.320752,0.000000,2342.946778,1206.039887,1572.148256,0.000000,0.000000
4,106558646,-25412.0,-32767.0,-32217.0,-27204.0,-10254.0,-10223.0,-15007.0,-14700.0,-30379.0,...,6626.926248,9987.517121,6586.279472,6676.764197,2435.131706,2966.428407,3244.105584,2942.466744,8078.247900,8753.817270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1265147,0.0,-2902.0,-8789.0,-2326.0,0.0,-1434.0,-2194.0,-1860.0,0.0,...,0.000000,486.778062,1714.410765,364.205085,0.000000,468.322480,323.412116,345.154588,0.000000,0.000000
726,115078097,-3924.0,-9948.0,-3716.0,-4771.0,-1192.0,-6078.0,-249.0,-358.0,-3389.0,...,611.803833,825.136816,509.310464,557.989619,184.299317,1662.581413,53.218984,66.156907,540.139436,849.049759
727,577108378,0.0,-2052.0,-4692.0,-2369.0,0.0,-1905.0,-1070.0,-1422.0,0.0,...,0.000000,450.901986,1424.694280,421.877082,0.000000,537.036483,299.404112,353.516468,0.000000,0.000000
728,108980580,-2973.0,0.0,-1816.0,-4907.0,-4735.0,-1170.0,-1643.0,0.0,-3476.0,...,280.260756,0.000000,242.735532,341.654619,239.321704,259.951368,233.648477,0.000000,347.004512,634.634993


In [46]:
# Saved the dataframe as a csv file
train_df.to_csv('train_df.csv',index=False)

In [47]:
# Saved the dataframe as a csv file
test_df.to_csv('test_df.csv',index=False)